# TFX Components Walk-through

The primary goal of this lab is to develop a high level understanding of TFX components.

You will utilize an experimental feature of TFX called *Interactive Context* to work with the TFX components interactivelly in a Jupyter notebook environment.

Working in an interactive notebook is useful when doing initial data exploriation, experimenting with models, and designing ML pipelines. You should be aware that there are differences in the way interactive notebooks are orchestrated, and how they access metadata artifacts.

In a production deployment of TFX you will use an orchestrator such as Kubeflow Pipelines, or Apache Beam. In an interactive notebook the notebook itself is the orchestrator, running each TFX component as you execute the notebook cells.

In a production deployment of TFX you will access metadata through the ML Metadata (MLMD) API. MLMD stores metadata properties in a database such as MySQL, and stores the metadata payloads in a persistent store such as Google Cloud Storage. In an interactive notebook, both properties and payloads are stored in the local file system of the Jupyter host.

You will work with the [Covertype Data Set](https://github.com/jarokaz/mlops-labs/blob/master/datasets/covertype/README.md) and use TFX  to analyze, understand and pre-process the dataset and train, analyze, validate and deploy the multi-class classification model.


The lab is designed to be instructor led. The instructor will walk you through the lab and provide commentary about each step. 

In [31]:
import absl
import os
import tempfile

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import BigQueryExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input


print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)

tf.enable_eager_execution()

Tensorflow Version: 1.15.0
TFX Version: 0.15.0


## Configure lab settings

Set location paths, connections strings, and other environment settings. Make sure to update project id, region, and artifact store GCS bucket with your settings.

In [2]:
PROJECT_ID = 'mlops-workshop'
ARTIFACT_STORE='gs://mlops-workshop-artifact-store'
REGION = 'us-central1'

PIPELINE_NAME='tfx-covertype-classifier'
PIPELINE_ROOT='{}/{}'.format('/home/tfx-interactive', PIPELINE_NAME)
os.makedirs(PIPELINE_ROOT, exist_ok=True)

DATA_ROOT = 'gs://workshop-datasets/covertype/full'

## TFX Component Walk-through

In the cells that follow you will construct TFX components and run each one interactively within the InteractiveContext to obtain ExecutionResult objects. This mirrors the process of an orchestrator running components in a TFX DAG based on when the dependencies for each component are met. 



### Create Interactive Context

TFX Interactive Context allows you to create and run TFX Components in an interactive mode. It is designed to support experimentation and development in a Jupyter Notebook environment. It is an experimental feature and major changes to interface and functionality are expected. When creating the interactive context you can specifiy the following parameters:
- `pipeline_name` - Optional name of the pipeline for ML Metadata tracking purposes. If not specified, a name will be generated for you.
- `pipeline_root` - Optional path to the root of the pipeline's outputs. If not specified, an ephemeral temporary directory will be created and used.
- `metadata_connection_config` - Optional `metadata_store_pb2.ConnectionConfig` instance used to configure connection to a ML Metadata connection. If not specified, an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.


In [3]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None
)

### Ingest data using ExampleGen

The `ExampleGen` component ingests data into a TFX pipeline. It consumes external files/services to generate Examples which will be read by other TFX components. It can also split and shuffle the data.

- Consumes: Data from external data sources such as CSV, TFRecord and BigQuery
- Emits: tf.Example records

#### Configure CsvExampleGen

In this exercise, you use the `CsvExampleGen` specialization of `ExampleGen` to ingest CSV files from GCS. The component is configured to split the input data into two splits - `train` and `eval` - using 4:1 ratio.  


In [4]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

example_gen = tfx.components.CsvExampleGen(
    instance_name='Data_Extraction_Spliting',
    input=external_input(DATA_ROOT),
    output_config=output_config
)

#### Run the CsvExampleGen component

In [5]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen.Data_Extraction_Spliting
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /home/tfx-interactive/tfx-covertype-classifier/CsvExampleGen.Data_Extraction_Spliting/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /home/tfx-interactive/tfx-covertype-classifier/CsvExampleGen.Data_Extraction_Spliting/examples/1/eval/, split: eval, id: 3)]
        ))

#### Examine the ingested data

In [8]:
train_uri = example_gen.outputs['examples'].get()[0].uri
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
decoder = tfdv.TFExampleDecoder()
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = decoder.decode(serialized_example)
  pprint(example)

{'Aspect': array([45]),
 'Cover_Type': array([1]),
 'Elevation': array([2778]),
 'Hillshade_3pm': array([143]),
 'Hillshade_9am': array([221]),
 'Hillshade_Noon': array([229]),
 'Horizontal_Distance_To_Fire_Points': array([6702]),
 'Horizontal_Distance_To_Hydrology': array([376]),
 'Horizontal_Distance_To_Roadways': array([2770]),
 'Slope': array([5]),
 'Soil_Type': array([b'C4744'], dtype=object),
 'Vertical_Distance_To_Hydrology': array([43]),
 'Wilderness_Area': array([b'Rawah'], dtype=object)}
{'Aspect': array([55]),
 'Cover_Type': array([2]),
 'Elevation': array([2992]),
 'Hillshade_3pm': array([114]),
 'Hillshade_9am': array([228]),
 'Hillshade_Noon': array([211]),
 'Horizontal_Distance_To_Fire_Points': array([511]),
 'Horizontal_Distance_To_Hydrology': array([120]),
 'Horizontal_Distance_To_Roadways': array([190]),
 'Slope': array([13]),
 'Soil_Type': array([b'C7755'], dtype=object),
 'Vertical_Distance_To_Hydrology': array([25]),
 'Wilderness_Area': array([b'Commanche'], dtype=

### Generate statistics using StatisticsGen

The `StatisticsGen`  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

- Consumes: datasets created by ExampleGen 
- Emits: Dataset statistics

`StatisticsGen` generate statistics for each split in the `ExampleGen` component's output. In our case there two splits: `train` and `eval`.

#### Run `StatisticsGen`

In [9]:
statistics_gen = tfx.components.StatisticsGen(
    instance_name='Statistics_Generation',
    examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen.Statistics_Generation
    execution_id: 2
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: /home/tfx-interactive/tfx-covertype-classifier/StatisticsGen.Statistics_Generation/output/2/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: /home/tfx-interactive/tfx-covertype-classifier/StatisticsGen.Statistics_Generation/output/2/eval/, split: eval, id: 5)]
        ))

#### Visualize statistics

In [18]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### Infer data schema using SchemaGen

Some TFX components use a description input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

- Consumes: statistics from `StatisticsGen`
- Emits: Data schema proto

`SchemaGen` generates the schema using the statistics for the `train` split. Statistics for other splits are ignored.

#### Run SchemaGen

In [19]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        output: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /home/tfx-interactive/tfx-covertype-classifier/SchemaGen/output/3/, split: , id: 6)]
        ))

#### Visualize the inferred schema

In [20]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Vertical_Distance_To_Hydrology',INT,required,single,-
'Aspect',INT,required,single,-
'Elevation',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Slope',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Soil_Type',STRING,required,single,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Update the schema

In most cases the auto-generated schemas must be fine tuned manually using insights from data exploration and/or domain knowledge about the data. For example, you know that in the `covertype` dataset there are seven types of forest cover (coded using 1-7 range) and that the value of the `Slope` feature should be in the 0-90 range. You can manually add these constraints to the auto-generated schema.



#### Load the auto-generated schema proto file

In [21]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

#### Modify the schema

In [22]:
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=1, max=7, is_categorical=True))
tfdv.set_domain(schema, 'Slope',  schema_pb2.IntDomain(name='Slope', min=0, max=90))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,single,"[1,7]"
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Vertical_Distance_To_Hydrology',INT,required,single,-
'Aspect',INT,required,single,-
'Elevation',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Slope',INT,required,single,"[0,90]"
'Hillshade_Noon',INT,required,single,-
'Soil_Type',STRING,required,single,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Save the updated schema 

In [23]:
schema_dir = '/home/covertype_schema'
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "Cover_Type"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  int_domain {
    name: "Cover_Type"
    min: 1
    max: 7
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Horizontal_Distance_To_Hydrology"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Vertical_Distance_To_Hydrology"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Aspect"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Elevation"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Horizontal_Distance_To_Roadways"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
 

### Import the updated schema using ImporterNode

The `ImporterNode` component allows you to import an external artifact, including the schema file, so it can be used by other TFX components in your workflow. 


#### Configure and run the ImporterNode component

In [24]:
schema_importer = ImporterNode(
    instance_name='Schema_Importer',
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False
)

context.run(schema_importer)

ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /home/covertype_schema, split: , id: 7)]
        ))

#### Visualize the imported schema

In [25]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,single,"[1,7]"
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Vertical_Distance_To_Hydrology',INT,required,single,-
'Aspect',INT,required,single,-
'Elevation',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-
'Slope',INT,required,single,"[0,90]"
'Hillshade_Noon',INT,required,single,-
'Soil_Type',STRING,required,single,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Validate data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema generated by `SchemaGen` or imported by `ImporterNode`.

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


- Consumes: A schema 
- Emits: Validation results

`ExampleValidator` validates the data in the `eval` split only. Other splits are ignored. 

#### Run ExampleValidator


In [26]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
    instance_name="Data_Validation"
)

context.run(example_validator, enable_cache=False)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 5
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /home/tfx-interactive/tfx-covertype-classifier/ExampleValidator.Data_Validation/output/5/, split: , id: 8)]
        ))

#### Example the output of `ExampleValidator`

The output artifact of the ExampleValidator is the `anomalies.pbtxt` file describing an anomalies_pb2.Anomalies protobuf.

In [36]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
anomalies_filename = os.path.join(train_uri, "anomalies.pbtxt")
!cat $anomalies_filename

baseline {
  feature {
    name: "Cover_Type"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    int_domain {
      name: "Cover_Type"
      min: 1
      max: 7
      is_categorical: true
    }
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Horizontal_Distance_To_Hydrology"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Vertical_Distance_To_Hydrology"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Aspect"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Elevation"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    

#### Visualize validation results

The file `anomalies.pbtxt` can be visualized using `context.show`.

In [37]:
context.show(example_validator.outputs['output'])

In our case no anomalies were detected in the `eval` split.

For a detailed deep dive into data validation and schema generation refer to the `lab-31-tfdv-structured-data` lab.

### Preprocess data with Transform

The `Transform` component performs feature engineering on `tf.Examples` emitted from an `ExampleGen` component, using a data schema created by `SchemaGen`  or imported by `ImporterNode`. When executed, `Transform` accepts `tf.Examples` emitted from the `ExampleGen` component and emits the transformed feature data and the `SavedModel` graph that was used to process the data. The emitted `SavedModel`  can then be used by serving components to make sure that the same data pre-processing logic is applied at training and serving.

- Consumes: tf.Examples and a data schema 
- Emits: A SavedModel and transformed data

The `Transform` component requires more code than many other components because of the arbitrary complexity of the feature engineering that you may need for the data and/or model that you're working with. It requires code files to be available which define the processing needed.

#### Define the pre-processing module

To configure `Trainsform`, you need to encapsulate your pre-processing code in the Python `preprocessing_fn` function and save it to a  python module that is then provided to the Transform component as an input. This module will be loaded by transform and the `preprocessing_fn` function will be called when the `Transform` component runs.

In most cases, your implementation of the `preprocessing_fn` makes extensive use of [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft) for performing feature engineering on your dataset.

In [38]:
_transform_module = 'covertype_transform.py'

In [39]:
%%writefile {_transform_module}

import tensorflow as tf
import tensorflow_transform as tft

NUMERIC_FEATURES_KEYS = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points']

CATEGORICAL_FEATURES_KEYS = ['Wilderness_Area', 'Soil_Type']

LABEL_KEY = 'Cover_Type'


def _transformed_name(key):
    return key + '_xf'

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.

    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.

    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
        in the second dimension.

    Returns:
      A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
        tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value),
        axis=1)

def preprocessing_fn(inputs):
    """Preprocesses Covertype Dataset.
    
    Scales numerical features and generates vocabularies
    and mappings for categorical features.
    
    Args:
        inputs: A map from feature keys to raw not-yet-transformed features
        
    Returns:
        A map from transformed feature keys to transformation operations
    """
    
    outputs = {}
    
    # Scale numerical features
    for key in NUMERIC_FEATURES_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))
        
    # Generate vocabularies and maps categorical features
    for key in CATEGORICAL_FEATURES_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        x=_fill_in_missing(inputs[key]),
        num_oov_buckets=1,
        vocab_filename=key)
        
        
    # Convert Cover_Type from 1-7 to 0-6 
    outputs[_transformed_name(LABEL_KEY)] = _fill_in_missing(inputs[LABEL_KEY]) - 1
    
    return outputs
    

Overwriting covertype_transform.py


#### Create and run the `Transform` component.

In [40]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=_transform_module)

context.run(transform, enable_cache=False)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/tfx-interactive/tfx-covertype-classifier/Transform/transform_output/6/.temp_path/tftransform_tmp/9032b4c00d2a4645ae9e111058edf33b/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/tfx-interactive/tfx-covertype-classifier/Transform/transform_output/6/.temp_path/tftransform_tmp/ca92569d961f4413a9462fe555affdb4/saved_model.pb
INFO:tensorflow:Saver not created because there are no variabl

ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: /home/tfx-interactive/tfx-covertype-classifier/Transform/transform_output/6/, split: , id: 9)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /home/tfx-interactive/tfx-covertype-classifier/Transform/transformed_examples/6/train/, split: train, id: 10)
            Artifact(type_name: ExamplesPath, uri: /home/tfx-interactive/tfx-covertype-classifier/Transform/transformed_examples/6/eval/, split: eval, id: 11)]
        ))

#### Examine `Transform` outputs

The Transform component has 2 outputs:

- `transform_output` - contains the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- `transformed_examples` - contains the preprocessed training and evaluation data.

Take a peek at the `transform_outpu` artifact: it points to a directory containing 3 subdirectories:

In [41]:
os.listdir(transform.outputs['transform_output'].get()[0].uri)

['transformed_metadata', 'transform_fn', 'metadata']

The transform_fn subdirectory contains the actual preprocessing graph. The metadata subdirectory contains the schema of the original data. The transformed_metadata subdirectory contains the schema of the preprocessed data.

The `transformed_examples` folder contains `tfrecord` files with transformed data.

In [42]:
transform_uri = transform.outputs['transformed_examples'].get()[1].uri
tfrecord_filenames = [os.path.join(transform_uri, name)
                      for name in os.listdir(transform_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
decoder = tfdv.TFExampleDecoder()
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = decoder.decode(serialized_example)
  pprint(example)

{'Aspect_xf': array([-0.4247225], dtype=float32),
 'Cover_Type_xf': array([0]),
 'Elevation_xf': array([0.4042254], dtype=float32),
 'Hillshade_3pm_xf': array([-1.6075127], dtype=float32),
 'Hillshade_9am_xf': array([1.413327], dtype=float32),
 'Hillshade_Noon_xf': array([-0.5728974], dtype=float32),
 'Horizontal_Distance_To_Fire_Points_xf': array([-0.5346177], dtype=float32),
 'Horizontal_Distance_To_Hydrology_xf': array([-0.28044012], dtype=float32),
 'Horizontal_Distance_To_Roadways_xf': array([-0.7065593], dtype=float32),
 'Slope_xf': array([0.78883255], dtype=float32),
 'Soil_Type_xf': array([3]),
 'Vertical_Distance_To_Hydrology_xf': array([-0.24732949], dtype=float32),
 'Wilderness_Area_xf': array([1])}
{'Aspect_xf': array([1.631343], dtype=float32),
 'Cover_Type_xf': array([1]),
 'Elevation_xf': array([-0.99347556], dtype=float32),
 'Hillshade_3pm_xf': array([0.77088565], dtype=float32),
 'Hillshade_9am_xf': array([-1.5381305], dtype=float32),
 'Hillshade_Noon_xf': array([-0.87

### Train with the `Trainer` component

The `Trainer` component trains a TensorFlow model.

`Trainer` takes:

- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- A data schema created by `SchemaGen` or imported by `ImporterNode`.
- A proto definition of train args and eval args.
- An optional transform graph produced by upstream Transform component.
- An optional base models used for scenarios such as warmstart.

Trainer emits: 
- A `SavedModel` and an `EvalSavedModel`

#### Define the trainer module

To configure `Trainer`, you need to encapsulate your training code in a Python module that is then provided to the `Trainer` as an input. The module must include the `trainer_fn` function that must return an estimator. f you prefer to work with Keras, you can do so and then convert the Keras model to an estimator using `tf.keras.model_to_estimator()`.


In [43]:
_trainer_module_file = 'covertype_trainer.py'

In [48]:
%%writefile {_trainer_module_file}

import tensorflow as tf

import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

NUMERIC_FEATURE_KEYS = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points']

CATEGORICAL_FEATURE_KEYS = ['Wilderness_Area', 'Soil_Type']

LABEL_KEY = 'Cover_Type'
NUM_CLASSES = 7

EXPORTED_MODEL_NAME='covertype-classifier'

# Define helper functions
def _transformed_name(key):
    return key + '_xf'

def _get_raw_feature_spec(schema):
  return schema_utils.schema_as_feature_spec(schema).feature_spec

def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')


def _build_estimator(config, numeric_feature_keys, categorical_feature_keys, hidden_units, warm_start_from=None):
  """Build an estimator for predicting forest cover based on cartographic data

  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.

  Returns:
    The estimator that will be used for training and eval.
  """

  num_feature_columns = [
      tf.feature_column.numeric_column(key)
      for key in numeric_feature_keys
  ]
  categorical_feature_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=num_buckets, default_value=0)
      for key, num_buckets in categorical_feature_keys
  ]
  
  return tf.estimator.DNNLinearCombinedClassifier(
      config=config,
      n_classes=NUM_CLASSES,
      linear_feature_columns=categorical_feature_columns,
      dnn_feature_columns=num_feature_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25],
      warm_start_from=warm_start_from)


def _input_fn(filenames, feature_specs, label_key, batch_size=200):
  """Generates features and labels for training or evaluation.

  Args:
    filenames: [str] list of CSV files to read data from.
    tf_transform_output: A TFTransformOutput.
    batch_size: int First dimension size of the Tensors returned by input_fn

  Returns:
    A (features, indices) tuple where features is a dictionary of
      Tensors, and indices is a single Tensor of label indices.
  """

  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=filenames, 
      batch_size=batch_size, 
      features=feature_specs,
      label_key=label_key,
      reader=_gzip_reader_fn)

  return dataset


def _example_serving_receiver_fn(tf_transform_output, schema, label_key):
  """Build the serving in inputs.

  Args:
    tf_transform_output: A TFTransformOutput.
    schema: the schema of the input data.

  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(label_key)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  transformed_features = tf_transform_output.transform_raw_features(
      serving_input_receiver.features)

  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)

def _eval_input_receiver_fn(tf_transform_output, schema, label_key):
  """Build everything needed for the tf-model-analysis to run the model.

  Args:
    tf_transform_output: A TFTransformOutput.
    schema: the schema of the input data.
    label_key: the name of the transformed label

  Returns:
    EvalInputReceiver function, which contains:
      - Tensorflow graph which parses raw untransformed features, applies the
        tf-transform preprocessing operators.
      - Set of raw, untransformed features.
      - Label against which predictions will be compared.
  """
  # Notice that the inputs are raw features, not transformed features here.
  raw_feature_spec = _get_raw_feature_spec(schema)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  features = serving_input_receiver.features.copy()
  transformed_features = tf_transform_output.transform_raw_features(features)
  
  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=features,
      receiver_tensors=serving_input_receiver.receiver_tensors,
      labels=transformed_features[label_key])


def trainer_fn(hparams, schema):
    
    train_batch_size = 40
    eval_batch_size = 40
    hidden_units = [128, 64]
    
    # Retrieve transformed feature specs
    tf_transform_output = tft.TFTransformOutput(hparams.transform_output)
    transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
    
    print(transformed_feature_spec)
    print(type(transformed_feature_spec))
    
    # Prepare transformed feature name lists
    # For categorical features retrieve vocabulary sizes
    transformed_label_key = _transformed_name(LABEL_KEY)
    transformed_numeric_feature_keys = [_transformed_name(key) for key in NUMERIC_FEATURE_KEYS]
    transformed_categorical_feature_keys = [
        (_transformed_name(key), tf_transform_output.num_buckets_for_transformed_feature(_transformed_name(key)))
         for key in CATEGORICAL_FEATURE_KEYS]
    
    # Create a training input function
    train_input_fn = lambda: _input_fn(
        filenames=hparams.train_files,
        feature_specs=tf_transform_output.transformed_feature_spec().copy(),
        batch_size=train_batch_size,
        label_key=transformed_label_key
    )
    
    # Create an evaluation input function
    eval_input_fn = lambda: _input_fn(
        filenames=hparams.eval_files,
        feature_specs=tf_transform_output.transformed_feature_spec().copy(),
        batch_size=eval_batch_size,
        label_key=transformed_label_key
    )
    
    # Create a training specification
    train_spec = tf.estimator.TrainSpec(
        train_input_fn,
        max_steps=hparams.train_steps
    )
    
    # Create an evaluation specifaction
    serving_receiver_fn = lambda: _example_serving_receiver_fn(
      tf_transform_output, schema, LABEL_KEY)
    exporter = tf.estimator.FinalExporter(EXPORTED_MODEL_NAME, serving_receiver_fn)
    
    eval_spec = tf.estimator.EvalSpec(
      eval_input_fn,
      steps=hparams.eval_steps,
      exporters=[exporter],
      name=EXPORTED_MODEL_NAME)
    
    # Create runtime config
    run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=999, keep_checkpoint_max=1)

    run_config = run_config.replace(model_dir=hparams.serving_model_dir)
    
    # Build an estimator
    estimator = _build_estimator(
        hidden_units=hidden_units,
        numeric_feature_keys=transformed_numeric_feature_keys, 
        categorical_feature_keys=transformed_categorical_feature_keys,
        config=run_config,
        warm_start_from=hparams.warm_start_from)
    
    # Create an input receiver for TFMA processing
    receiver_fn = lambda: _eval_input_receiver_fn(
      tf_transform_output, schema, transformed_label_key)
    
    return {
      'estimator': estimator,
      'train_spec': train_spec,
      'eval_spec': eval_spec,
      'eval_input_receiver_fn': receiver_fn
    }
    


Overwriting covertype_trainer.py


#### Create and run the Trainer component

In [49]:
trainer = Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_output=transform.outputs['transform_output'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))

context.run(trainer)

{'Elevation_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Horizontal_Distance_To_Fire_Points_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Vertical_Distance_To_Hydrology_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Horizontal_Distance_To_Roadways_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Hillshade_Noon_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Soil_Type_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None), 'Wilderness_Area_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None), 'Aspect_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Hillshade_3pm_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Cover_Type_xf': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None), 'Horizontal_Distance_To_Hydrology_xf': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'Hillshade

ExecutionResult(
    component_id: Trainer
    execution_id: 8
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: /home/tfx-interactive/tfx-covertype-classifier/Trainer/output/8/, split: , id: 13)]
        ))

### Analyze training with TensorBoard

In this step you will analyze the training run with [TensorBoard.dev](https://blog.tensorflow.org/2019/12/introducing-tensorboarddev-new-way-to.html). `TensorBoard.dev` is a managed service that enables you to easily host, track and share your ML experiments.

#### Retrieve the location of TensorBoard logs

In [53]:
train_uri = trainer.outputs['model'].get()[0].uri
logs_path = os.path.join(train_uri, 'serving_model_dir')
print(logs_path)

/home/tfx-interactive/tfx-covertype-classifier/Trainer/output/8/serving_model_dir


#### Upload the logs and start TensorBoard.dev

In [ ]:
!tensorboard dev upload --logdir $logs_path

Upload started and will continue reading any new data as it's added
to the logdir. To stop uploading, press Ctrl-C.
View your TensorBoard live at: https://tensorboard.dev/experiment/OYG7eclkTdOz0aDkQORRxA/
